## 基于Pyecharts的疫情数据可视化

本文使用的数据源来自https://lab.isaaclin.cn/nCoV/zh；

本接口为全国新型肺炎疫情实时数据接口，数据来源为丁香园，开放给所有有需要的人。

- 项目地址	https://github.com/BlankerL/DXY-COVID-19-Crawler	

- 数据仓库	https://github.com/BlankerL/DXY-COVID-19-Data
	



In [1]:
import requests
import pyecharts
from pyecharts.charts import *
from pyecharts import options
from pyecharts.commons.utils import JsCode
from datetime import date,datetime
import datetime
import warnings
warnings.filterwarnings("ignore")





## 数据来源
- 通过https://lab.isaaclin.cn/nCoV/api/area接口获取最新疫情数据；

- 通过https://lab.isaaclin.cn/nCoV/api/area?latest=0接口获取时间序列疫情数据；

- 通过http://lab.isaaclin.cn/nCoV/api/overall?latest=0接口获取全国时间序列疫情数据；

In [5]:
# 数据来源于 Github DXY-COVID-19-Data， 原作者为 Isaac Lin
url = 'https://lab.isaaclin.cn/nCoV/api/area'
data = requests.get(url).json()
print(data)

shName': 'Zhuzhou'}, {'cityName': '娄底', 'currentConfirmedCount': 0, 'confirmedCount': 76, 'suspectedCount': 0, 'curedCount': 76, 'deadCount': 0, 'locationId': 431300, 'cityEnglishName': 'Loudi'}, {'cityName': '益阳', 'currentConfirmedCount': 0, 'confirmedCount': 60, 'suspectedCount': 0, 'curedCount': 60, 'deadCount': 0, 'locationId': 430900, 'cityEnglishName': 'Yiyang'}, {'cityName': '衡阳', 'currentConfirmedCount': 0, 'confirmedCount': 48, 'suspectedCount': 0, 'curedCount': 48, 'deadCount': 0, 'locationId': 430400, 'cityEnglishName': 'Hengyang'}, {'cityName': '永州', 'currentConfirmedCount': 0, 'confirmedCount': 44, 'suspectedCount': 0, 'curedCount': 44, 'deadCount': 0, 'locationId': 431100, 'cityEnglishName': 'Yongzhou'}, {'cityName': '怀化', 'currentConfirmedCount': 0, 'confirmedCount': 40, 'suspectedCount': 0, 'curedCount': 40, 'deadCount': 0, 'locationId': 431200, 'cityEnglishName': 'Huaihua'}, {'cityName': '郴州', 'currentConfirmedCount': 0, 'confirmedCount': 39, 'suspectedCount': 0, 'cure

In [6]:
# 生成更新日期
update_date = date.today()
print(update_date)

2020-03-20


In [12]:
oversea_data = {'currentConfirmedCount':{'国内':0, '海外':0},
                'confirmedCount':{'国内':0, '海外':0},
                'curedCount':{'国内':0, '海外':0},
                'deadCount':{'国内':0, '海外':0}}
for item in data['results']:
    if item['provinceEnglishName'] == 'China':
        oversea_data['currentConfirmedCount']['国内']+=item['currentConfirmedCount']
        oversea_data['confirmedCount']['国内']+=item['confirmedCount']
        oversea_data['curedCount']['国内']+=item['curedCount']
        oversea_data['deadCount']['国内']+=item['deadCount']
    elif item['countryEnglishName'] != 'China':
        oversea_data['currentConfirmedCount']['海外']+=item['currentConfirmedCount']
        oversea_data['confirmedCount']['海外']+=item['confirmedCount']
        oversea_data['curedCount']['海外']+=item['curedCount']
        oversea_data['deadCount']['海外']+=item['deadCount']

fn = """
    function(params) {
        if(params.name == '海外')
            return '\\n\\n\\n' + params.name + ' : ' + params.value;
        return params.seriesName + '\\n' + params.name + ' : ' + params.value;
    }
    """


def new_label_opts():
    return pyecharts.options.LabelOpts(formatter=JsCode(fn), position="center")


pie = (Pie(init_opts=pyecharts.options.InitOpts(theme='dark', width='400'))
    .add(
        "当前确诊",
        [(x, y) for x, y in oversea_data['currentConfirmedCount'].items()],
        center=["30%", "30%"],
        radius=[60, 90],
        label_opts=new_label_opts(),
    )
    .add(
        "累计确诊",
        [(x, y) for x, y in oversea_data['confirmedCount'].items()],
        center=["70%", "30%"],
        radius=[60, 90],
        label_opts=new_label_opts(),
    )
    .add(
        "治愈",
        [(x, y) for x, y in oversea_data['curedCount'].items()],
        center=["30%", "75%"],
        radius=[60, 90],
        label_opts=new_label_opts(),
    )
    .add(
        "死亡",
        [(x, y) for x, y in oversea_data['deadCount'].items()],
        center=["70%", "75%"],
        radius=[60, 90],
        label_opts=new_label_opts(),
    )
    .set_global_opts(
        title_opts=pyecharts.options.TitleOpts(title="海内外疫情数据",
                                  subtitle="更新日期：{}".format(update_date)),
        legend_opts=pyecharts.options.LegendOpts(is_show=True),)
    .set_series_opts(
        tooltip_opts=pyecharts.options.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),)
    )

pie.render_notebook()

### 全球疫情地图

In [15]:
oversea_confirm = []
for item in data['results']:
    if item['provinceEnglishName']:
        oversea_confirm.append((item['provinceEnglishName'].replace('United States of America', 'United States')
                                                          .replace('United Kiongdom', 'United Kingdom'), 
                                item['currentConfirmedCount']))
                                
_map = (
        Map(init_opts=pyecharts.options.InitOpts(theme='dark', width='400'))
        .add("当前确诊人数", oversea_confirm, "world",is_map_symbol_show=False,  is_roam=False)
        .set_series_opts(label_opts=pyecharts.options.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=pyecharts.options.TitleOpts(title="新型冠状病毒全球疫情地图",
                                     subtitle="更新日期：{}".format(update_date)),
            legend_opts=pyecharts.options.LegendOpts(is_show=False),
            visualmap_opts=pyecharts.options.VisualMapOpts(is_show=True, max_=1000, 
                                              is_piecewise=False, 
                                              range_color=['#FFFFE0', '#8B0000']),
            graphic_opts=[
                    pyecharts.options.GraphicGroup(
                        graphic_item=pyecharts.options.GraphicItem(
                            bounding="raw",
                            right=150,
                            bottom=50,
                            z=100,
                        ),
                        children=[
                            pyecharts.options.GraphicRect(
                                graphic_item=pyecharts.options.GraphicItem(
                                    left="center", top="center", z=100
                                ),
                                graphic_shape_opts=pyecharts.options.GraphicShapeOpts(
                                    width=200, height=50
                                ),
                                graphic_basicstyle_opts=pyecharts.options.GraphicBasicStyleOpts(
                                    fill="rgba(0,0,0,0.3)"
                                ),
                            ),
                            pyecharts.options.GraphicText(
                                graphic_item=pyecharts.options.GraphicItem(
                                    left="center", top="center", z=100
                                ),
                                graphic_textstyle_opts=pyecharts.options.GraphicTextStyleOpts(
                                    text=JsCode("['钻石号邮轮', '当前确诊人数：{}人'].join('\\n')"
                                                .format(dict(oversea_confirm)['Diamond Princess Cruise Ship'])),
                                    font="bold 16px Microsoft YaHei",
                                    graphic_basicstyle_opts=pyecharts.options.GraphicBasicStyleOpts(
                                        fill="#fff"
                                    ),
                                ),
                            ),
                        ],
                    )
                ],
        )
    )

_map.render_notebook()

### 全国疫情地图

In [22]:
cofirm, currentCofirm, cured, dead = [], [], [], []
for item in data['results']:
    if item['countryName'] == '中国':
        cofirm.append((item['provinceShortName'], item['confirmedCount']))
        currentCofirm.append((item['provinceShortName'], item['currentConfirmedCount']))
        cured.append((item['provinceShortName'], item['curedCount']))
        dead.append((item['provinceShortName'], item['deadCount']))

tab = Tab()

_map = (
        Map(init_opts=pyecharts.options.InitOpts(theme='dark'))
        .add("累计确诊人数", cofirm, "china",is_map_symbol_show=False,  is_roam=False)
        .set_series_opts(label_opts=pyecharts.options.LabelOpts(is_show=True))
        .set_global_opts(
            title_opts=pyecharts.options.TitleOpts(title="新型冠状病毒全国疫情地图",
                                     subtitle="更新日期：{}".format(update_date)),
            legend_opts=pyecharts.options.LegendOpts(is_show=False),
            visualmap_opts=pyecharts.options.VisualMapOpts(is_show=True, max_=1000, 
                                              is_piecewise=False, 
                                              range_color=['#FFFFE0', '#FFA07A', '#CD5C5C', '#8B0000'])
        )
)
tab.add(_map, '累计确诊')

_map = (
        Map(init_opts=pyecharts.options.InitOpts(theme='dark'))
        .add("当前确诊人数", currentCofirm, "china",is_map_symbol_show=False,  is_roam=False)
        .set_series_opts(label_opts=pyecharts.options.LabelOpts(is_show=True))
        .set_global_opts(
            title_opts=pyecharts.options.TitleOpts(title="新型冠状病毒全国疫情地图",
                                     subtitle="更新日期：{}".format(update_date)),
            legend_opts=pyecharts.options.LegendOpts(is_show=False),
            visualmap_opts=pyecharts.options.VisualMapOpts(is_show=True, max_=100, 
                                              is_piecewise=False, 
                                              range_color=['#FFFFE0', '#FFA07A', '#CD5C5C', '#8B0000'])
        )
)
tab.add(_map, '当前确诊')

_map = (
        Map(init_opts=pyecharts.options.InitOpts(theme='dark'))
        .add("治愈人数", cured, "china",is_map_symbol_show=False,  is_roam=False)
        .set_series_opts(label_opts=pyecharts.options.LabelOpts(is_show=True))
        .set_global_opts(
            title_opts=pyecharts.options.TitleOpts(title="新型冠状病毒全国疫情地图",
                                     subtitle="更新日期：{}".format(update_date)),
            legend_opts=pyecharts.options.LegendOpts(is_show=False),
            visualmap_opts=pyecharts.options.VisualMapOpts(is_show=True, max_=1000, 
                                              is_piecewise=False, 
                                              range_color=['#FFFFE0', 'green'])
        )
)
tab.add(_map, '治愈')

_map = (
        Map(init_opts=pyecharts.options.InitOpts(theme='dark'))
        .add("死亡人数", dead, "china",is_map_symbol_show=False,  is_roam=False)
        .set_series_opts(label_opts=pyecharts.options.LabelOpts(is_show=True))
        .set_global_opts(
            title_opts=pyecharts.options.TitleOpts(title="新型冠状病毒全国疫情地图",
                                     subtitle="更新日期：{}".format(update_date)),
            legend_opts=pyecharts.options.LegendOpts(is_show=False),
            visualmap_opts=pyecharts.options.VisualMapOpts(is_show=True, max_=50, 
                                              is_piecewise=False, 
                                              range_color=['#FFFFE0', '#FFA07A', '#CD5C5C', '#8B0000'])
        )
)
tab.add(_map, '死亡')

tab.render_notebook()

## 湖北省内确诊情况

In [26]:
for item in data['results']:
    if item['provinceShortName'] == '湖北':
        hubei_data = item['cities']
        
bar = (
        Bar(init_opts=pyecharts.options.InitOpts(theme='dark', width='400'))
        .add_xaxis([x['cityName'] for x in hubei_data])
        .add_yaxis("治愈", [x['curedCount'] for x in hubei_data], stack="stack1",
                  itemstyle_opts=pyecharts.options.ItemStyleOpts(
                                border_color='rgb(220,220,220)',
                                color=JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, 
                                             [{
                                                 offset: 0,
                                                 color: 'rgb(255,215,0)'
                                             }, {
                                                 offset: 1,
                                                 color: 'rgb(32,178,170)'
                                             }])""")))
        .add_yaxis("当前确诊", [x['currentConfirmedCount'] for x in hubei_data], stack="stack1",
                  itemstyle_opts=pyecharts.options.ItemStyleOpts(border_color='rgb(220,220,220)',
                                color=JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, 
                                             [{
                                                 offset: 0,
                                                 color: 'rgb(255,99,71)'
                                             }, {
                                                 offset: 1,
                                                 color: 'rgb(255,215,0)'
                                             }])""")))
        .add_yaxis("死亡", [x['deadCount'] for x in hubei_data], stack="stack1",
                  itemstyle_opts=pyecharts.options.ItemStyleOpts(border_color='rgb(220,220,220)',
                                color=JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, 
                                             [{
                                                 offset: 0,
                                                 color: 'rgb(128,0,0)'
                                             }, {
                                                 offset: 1,
                                                 color: 'rgb(255,99,71)'
                                             }])""")))
        .set_series_opts(label_opts=pyecharts.options.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=pyecharts.options.TitleOpts(title="新型冠状病毒湖北省内确诊情况",
                                     subtitle="更新日期：{}".format(update_date)),
            xaxis_opts=pyecharts.options.AxisOpts(axislabel_opts=pyecharts.options.LabelOpts(rotate=45)),
            legend_opts=pyecharts.options.LegendOpts(is_show=True),
            graphic_opts=[
                    pyecharts.options.GraphicGroup(
                        graphic_item=pyecharts.options.GraphicItem(
                            bounding="raw",
                            right=200,
                            top=120
                        ),
                        children=[
                            pyecharts.options.GraphicRect(
                                graphic_item=pyecharts.options.GraphicItem(
                                    left="center", top="center"
                                ),
                                graphic_shape_opts=pyecharts.options.GraphicShapeOpts(
                                    width=200, height=60
                                ),
                                graphic_basicstyle_opts=pyecharts.options.GraphicBasicStyleOpts(
                                    fill="rgba(0,0,0,0.3)"
                                ),
                            ),
                            pyecharts.options.GraphicText(
                                graphic_item=pyecharts.options.GraphicItem(
                                    left="center", top="center", z=1
                                ),
                                graphic_textstyle_opts=pyecharts.options.GraphicTextStyleOpts(
                                    text=JsCode("['当前确诊人数：', '','累计确诊人数-死亡人数-治愈人数'].join('\\n')"),
                                    font="bold 12px Microsoft YaHei",
                                    graphic_basicstyle_opts=pyecharts.options.GraphicBasicStyleOpts(
                                        fill="#fff"
                                    ),
                                ),
                            ),
                        ],
                    )
                ],
                )
        )

bar.render_notebook()

感谢https://www.kesci.com/home/project/5e50e7f50e2b66002c2057b3/comment